In [ ]:
# # This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive')

# # TODO: Enter the foldername in your Drive where you have saved the unzipped
# # assignment folder, e.g. 'cs6353/assignments/assignment2/'
# FOLDERNAME = 'assignment2'
# assert FOLDERNAME is not None, "[!] Enter the foldername."

# # Now that we've mounted your Drive, this ensures that
# # the Python interpreter of the Colab VM can load
# # python files from within it.
# import sys
# sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

# This downloads the CIFAR-10 dataset to your Drive
# if it doesn't already exist.
%cd /content/drive/MyDrive/MS/assignment2/cs6353/datasets
!bash get_datasets.sh
%cd ../..

# Install requirements from colab_requirements.txt
# TODO: Please change your path below to the colab_requirements.txt file
# ! python -m pip install -r /content/drive/My\ Drive/$FOLDERNAME/colab_requirements.txt

Mounted at /content/drive
/content/drive/MyDrive/MS/assignment2/cs6353/datasets
--2024-09-30 02:33:31--  http://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
Resolving www.cs.toronto.edu (www.cs.toronto.edu)... 128.100.3.30
Connecting to www.cs.toronto.edu (www.cs.toronto.edu)|128.100.3.30|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 170498071 (163M) [application/x-gzip]
Saving to: ‘cifar-10-python.tar.gz’

cifar-10-python.tar 100%[===================>] 162.60M  52.5MB/s    in 3.2s    

2024-09-30 02:33:34 (50.4 MB/s) - ‘cifar-10-python.tar.gz’ saved [170498071/170498071]

cifar-10-batches-py/
cifar-10-batches-py/data_batch_4
cifar-10-batches-py/readme.html
cifar-10-batches-py/test_batch
cifar-10-batches-py/data_batch_3
cifar-10-batches-py/batches.meta
cifar-10-batches-py/data_batch_2
cifar-10-batches-py/data_batch_5
cifar-10-batches-py/data_batch_1
/content/drive/MyDrive/MS/assignment2


# Softmax exercise

*Complete and hand in this completed worksheet (including its outputs and any supporting code outside of the worksheet) with your assignment submission. For more details see the [assignments page](https://utah.instructure.com/courses/919972/assignments/12590082) on the course website.*

This exercise is analogous to the SVM exercise. You will:

- implement a fully-vectorized **loss function** for the Softmax classifier
- implement the fully-vectorized expression for its **analytic gradient**
- **check your implementation** with numerical gradient
- use a validation set to **tune the learning rate and regularization** strength
- **optimize** the loss function with **SGD**
- **visualize** the final learned weights


In [ ]:
from __future__ import print_function
import random
import numpy as np
from cs6353.data_utils import load_CIFAR10
import matplotlib.pyplot as plt


%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [ ]:
def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=1000, num_dev=500):
    """
    Load the CIFAR-10 dataset from disk and perform preprocessing to prepare
    it for the linear classifier. These are the same steps as we used for the
    SVM, but condensed to a single function.
    """
    # Load the raw CIFAR-10 data
    cifar10_dir = 'cs6353/datasets/cifar-10-batches-py'

    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)

    # subsample the data
    mask = list(range(num_training, num_training + num_validation))
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = list(range(num_training))
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = list(range(num_test))
    X_test = X_test[mask]
    y_test = y_test[mask]
    mask = np.random.choice(num_training, num_dev, replace=False)
    X_dev = X_train[mask]
    y_dev = y_train[mask]

    # Preprocessing: reshape the image data into rows
    X_train = np.reshape(X_train, (X_train.shape[0], -1))
    X_val = np.reshape(X_val, (X_val.shape[0], -1))
    X_test = np.reshape(X_test, (X_test.shape[0], -1))
    X_dev = np.reshape(X_dev, (X_dev.shape[0], -1))

    # Normalize the data: subtract the mean image
    mean_image = np.mean(X_train, axis = 0)
    X_train -= mean_image
    X_val -= mean_image
    X_test -= mean_image
    X_dev -= mean_image

    # add bias dimension and transform into columns
    X_train = np.hstack([X_train, np.ones((X_train.shape[0], 1))])
    X_val = np.hstack([X_val, np.ones((X_val.shape[0], 1))])
    X_test = np.hstack([X_test, np.ones((X_test.shape[0], 1))])
    X_dev = np.hstack([X_dev, np.ones((X_dev.shape[0], 1))])

    return X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev


# Cleaning up variables to prevent loading data multiple times (which may cause memory issue)
try:
   del X_train, y_train
   del X_test, y_test
   print('Clear previously loaded data.')
except:
   pass

# Invoke the above function to get our data.
X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev = get_CIFAR10_data()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)
print('dev data shape: ', X_dev.shape)
print('dev labels shape: ', y_dev.shape)

Train data shape:  (49000, 3073)
Train labels shape:  (49000,)
Validation data shape:  (1000, 3073)
Validation labels shape:  (1000,)
Test data shape:  (1000, 3073)
Test labels shape:  (1000,)
dev data shape:  (500, 3073)
dev labels shape:  (500,)


## Softmax Classifier

Your code for this section will all be written inside **cs6353/classifiers/softmax.py**.


In [ ]:
# First implement the naive softmax loss function with nested loops.
# Open the file cs6353/classifiers/softmax.py and implement the
# softmax_loss_naive function.

from cs6353.classifiers.softmax import softmax_loss_naive
import time

# Generate a random softmax weight matrix and use it to compute the loss.
W = np.random.randn(3073, 10) * 0.0001
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 0.0)

# As a rough sanity check, our loss should be something close to -log(0.1).
print('loss: %f' % loss)
print('sanity check: %f' % (-np.log(0.1)))

loss: 2.342099
sanity check: 2.302585


## Inline Question 1:
Why do we expect our loss to be close to -log(0.1)? Explain briefly.**

**Your answer:**

Because the model has not been trained and learned anything. In other words, the model's predictions are random and hence making the score for each class having roughly the same probability. Since there are 10 classes, then the probability for each class is 0.1, therefore our loss is close to -log(0.1).

In [ ]:
# Complete the implementation of softmax_loss_naive and implement a (naive)
# version of the gradient that uses nested loops.
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 0.0)

# As we did for the SVM, use numeric gradient checking as a debugging tool.
# The numeric gradient should be close to the analytic gradient.
from cs6353.gradient_check import grad_check_sparse
f = lambda w: softmax_loss_naive(w, X_dev, y_dev, 0.0)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

# similar to SVM case, do another gradient check with regularization
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 5e1)
f = lambda w: softmax_loss_naive(w, X_dev, y_dev, 5e1)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

numerical: 3.565006 analytic: 3.565006, relative error: 1.043934e-08
numerical: 1.134127 analytic: 1.134127, relative error: 5.403774e-08
numerical: 2.474679 analytic: 2.474679, relative error: 1.827674e-08
numerical: -4.153848 analytic: -4.153849, relative error: 1.199399e-08
numerical: 3.818440 analytic: 3.818440, relative error: 4.730559e-09
numerical: 1.621642 analytic: 1.621641, relative error: 2.622742e-08
numerical: 0.807105 analytic: 0.807105, relative error: 1.128870e-08
numerical: 0.165301 analytic: 0.165301, relative error: 3.688017e-07
numerical: -0.061166 analytic: -0.061166, relative error: 6.045153e-07
numerical: -0.256459 analytic: -0.256459, relative error: 3.185238e-09
numerical: -3.257964 analytic: -3.257964, relative error: 1.567313e-08
numerical: 0.965306 analytic: 0.965306, relative error: 1.225586e-08
numerical: -1.138860 analytic: -1.138860, relative error: 1.989816e-08
numerical: 0.386250 analytic: 0.386249, relative error: 1.267685e-07
numerical: -0.230471 ana

In [ ]:
# Now that we have a naive implementation of the softmax loss function and its gradient,
# implement a vectorized version in softmax_loss_vectorized.
# The two versions should compute the same results, but the vectorized version should be
# much faster.
tic = time.time()
loss_naive, grad_naive = softmax_loss_naive(W, X_dev, y_dev, 0.000005)
toc = time.time()
print('naive loss: %e computed in %fs' % (loss_naive, toc - tic))

from cs6353.classifiers.softmax import softmax_loss_vectorized
tic = time.time()
loss_vectorized, grad_vectorized = softmax_loss_vectorized(W, X_dev, y_dev, 0.000005)
toc = time.time()
print('vectorized loss: %e computed in %fs' % (loss_vectorized, toc - tic))

# As we did for the SVM, we use the Frobenius norm to compare the two versions
# of the gradient.
grad_difference = np.linalg.norm(grad_naive - grad_vectorized, ord='fro')
print('Loss difference: %f' % np.abs(loss_naive - loss_vectorized))
print('Gradient difference: %f' % grad_difference)

naive loss: 2.342099e+00 computed in 0.320574s
vectorized loss: 2.342099e+00 computed in 0.017554s
Loss difference: 0.000000
Gradient difference: 0.000000


In [ ]:
# Use the validation set to tune hyperparameters (regularization strength and
# learning rate). You should experiment with different ranges for the learning
# rates and regularization strengths; if you are careful you should be able to
# get a classification accuracy of over 0.35 on the validation set.
from cs6353.classifiers import Softmax
results = {}
best_val = -1
best_softmax = None
# learning_rates = [1e-7, 5e-7]
# # regularization_strengths = [2.5e4, 5e4]
# learning_rates = np.linspace(2.5e-6, 2.7e-6, num=10)
# # learning_rates = [2.666666666666667e-06]
# regularization_strengths = np.linspace(4.5e4, 5.5e4, num=10)
# # regularization_strengths = [52413.793103448275]

learning_rates = np.linspace(2.1e-6, 2.5e-6, num=10)
regularization_strengths = np.linspace(4.5e4, 5.2e4, num=10)

################################################################################
# TODO:                                                                        #
# Use the validation set to set the learning rate and regularization strength. #
# This should be identical to the validation that you did for the SVM; save    #
# the best trained softmax classifier in best_softmax.                          #
################################################################################
lr_reg = []
for l_r in learning_rates:
    for reg in regularization_strengths:
        lr_reg.append((l_r, reg))

for l_r in lr_reg:
    softmax = Softmax()
    softmax.train(X_train, y_train, l_r[0], l_r[1], num_iters=750, batch_size=200, verbose=False)
    y_train_pred = softmax.predict(X_train)
    y_val_pred = softmax.predict(X_val)
    train_accuracy = np.mean(y_train == y_train_pred)
    val_accuracy = np.mean(y_val == y_val_pred)
    if val_accuracy > best_val:
        best_val = val_accuracy
        best_softmax = softmax
        best_l_r = l_r
    results[l_r] = (train_accuracy, val_accuracy)
################################################################################
#                              END OF YOUR CODE                                #
################################################################################

# Print out results.
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print('lr %e reg %e train accuracy: %f val accuracy: %f' % (
                lr, reg, train_accuracy, val_accuracy))

print('best validation accuracy achieved during cross-validation: %f' % best_val)
print('best learning rate and regularization strength', best_l_r)

lr 2.100000e-06 reg 4.500000e+04 train accuracy: 0.284694 val accuracy: 0.280000
lr 2.100000e-06 reg 4.577778e+04 train accuracy: 0.270490 val accuracy: 0.279000
lr 2.100000e-06 reg 4.655556e+04 train accuracy: 0.279612 val accuracy: 0.269000
lr 2.100000e-06 reg 4.733333e+04 train accuracy: 0.276571 val accuracy: 0.283000
lr 2.100000e-06 reg 4.811111e+04 train accuracy: 0.272653 val accuracy: 0.268000
lr 2.100000e-06 reg 4.888889e+04 train accuracy: 0.298837 val accuracy: 0.309000
lr 2.100000e-06 reg 4.966667e+04 train accuracy: 0.257184 val accuracy: 0.267000
lr 2.100000e-06 reg 5.044444e+04 train accuracy: 0.279143 val accuracy: 0.300000
lr 2.100000e-06 reg 5.122222e+04 train accuracy: 0.283061 val accuracy: 0.281000
lr 2.100000e-06 reg 5.200000e+04 train accuracy: 0.291347 val accuracy: 0.312000
lr 2.144444e-06 reg 4.500000e+04 train accuracy: 0.294388 val accuracy: 0.310000
lr 2.144444e-06 reg 4.577778e+04 train accuracy: 0.266490 val accuracy: 0.277000
lr 2.144444e-06 reg 4.655556

best validation accuracy achieved during cross-validation: 0.242000
best learning rate and regularization strength (2.8e-06, 100000.0)

best validation accuracy achieved during cross-validation: 0.312000
best learning rate and regularization strength (2.689655172413793e-06, 44137.93103448275)

best validation accuracy achieved during cross-validation: 0.302000
best learning rate and regularization strength (2.8e-06, 62758.620689655174)

best validation accuracy achieved during cross-validation: 0.326000
best learning rate and regularization strength (2.666666666666667e-06, 52413.793103448275)

best validation accuracy achieved during cross-validation: 0.296000
best learning rate and regularization strength (2.666666666666667e-06, 56206.89655172414)

best validation accuracy achieved during cross-validation: 0.295000
best learning rate and regularization strength (2.666666666666667e-06, 50000.0)

best validation accuracy achieved during cross-validation: 0.301000
best learning rate and regularization strength (2.666666666666667e-06, 53888.88888888889)

best validation accuracy achieved during cross-validation: 0.304000
best learning rate and regularization strength (2.666666666666667e-06, 50555.555555555555)

best validation accuracy achieved during cross-validation: 0.314000
best learning rate and regularization strength (2.6111111111111113e-06, 50555.555555555555)

best validation accuracy achieved during cross-validation: 0.318000
best learning rate and regularization strength (2.6777777777777776e-06, 45000.0)

best validation accuracy achieved during cross-validation: 0.331000
best learning rate and regularization strength (2.4444444444444447e-06, 48888.88888888889)

best validation accuracy achieved during cross-validation: 0.314000
best learning rate and regularization strength (2.111111111111111e-06, 51111.11111111111)

best validation accuracy achieved during cross-validation: 0.322000
best learning rate and regularization strength (2.144444444444444e-06, 46111.11111111111)

best validation accuracy achieved during cross-validation: 0.331000
best learning rate and regularization strength (2.3666666666666667e-06, 52000.0)

In [ ]:
# evaluate on test set
# Evaluate the best softmax on test set
y_test_pred = best_softmax.predict(X_test)
test_accuracy = np.mean(y_test == y_test_pred)
print('softmax on raw pixels final test set accuracy: %f' % (test_accuracy, ))

softmax on raw pixels final test set accuracy: 0.307000


**Inline Question** - *True or False*

It's possible to add a new data point to a training set that would leave the SVM loss unchanged, but this is not the case with the Softmax classifier loss.

*Your answer*:

**True**

*Your explanation*:

Because Softmax loss needs to sum all scores of all the classes, hence a new data points affects the value. Whereas SVM just takes the max between 0 and the difference of the scores which may have the chance that the new data point does not change the max value.

In [ ]:
# Visualize the learned weights for each class
w = best_softmax.W[:-1,:] # strip out the bias
w = w.reshape(32, 32, 3, 10)

w_min, w_max = np.min(w), np.max(w)

classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
for i in range(10):
    plt.subplot(2, 5, i + 1)

    # Rescale the weights to be between 0 and 255
    wimg = 255.0 * (w[:, :, :, i].squeeze() - w_min) / (w_max - w_min)
    plt.imshow(wimg.astype('uint8'))
    plt.axis('off')
    plt.title(classes[i])

In [ ]:
!jupyter nbconvert --to html /content/drive/MyDrive/MS/assignment2/softmax.ipynb

[NbConvertApp] Converting notebook /content/drive/MyDrive/MS/assignment2/softmax.ipynb to html
[NbConvertApp] Writing 636293 bytes to /content/drive/MyDrive/MS/assignment2/softmax.html
